1. Content base Filtering
   Berdasarkan karakteristik dari data(content) / tidak ada interksi dari usernya
   
    algorithms are given user preferences for items and recommend similar items based on a domain-specific notion of item content. This approach also extends naturally to cases where item metadata is available (e.g., movie stars, book authors, and music genres).

   A recommender system has to decide between two methods for information delivery when providing the user with recommendations:
- 
Exploitation. The system chooses documents similar to those for which the user has already expressed a preference- .
Exploration. The system chooses documents where the user profile does not provide evidence to predict the user’s reacti

References : https://fritz.ai/recommender-systems-with-python/on.

In [12]:
data = [
    {
        "product_id": 1,
        "product_name": "samsung galaxy s21",
        "tags": "HP, samsung, handphone"
    },
    {
        "product_id": 2,
        "product_name": "iphone 12",
        "tags": "HP, apple, handphone"
    },
    {
        "product_id": 3,
        "product_name": "google pixel 6",
        "tags": "HP, google, handphone"
    },
    {
        "product_id": 4,
        "product_name": "xiaomi mi 11",
        "tags": "HP, xiaomi, handphone"
    },
    {
        "product_id": 5,
        "product_name": "oneplus 9",
        "tags": "HP, oneplus, handphone"
    },
    {
        "product_id": 6,
        "product_name": "Laptop Lenovo Ideapad 3",
        "tags": "laptop, lenovo, ideapad"
    },
    {
        "product_id": 7,
        "product_name": "HP Spectre x360",
        "tags": "laptop, HP, spectre"
    },
    {
        "product_id": 8,
        "product_name": "mesin cuci samsung 8kg",
        "tags": "mesin cuci, samsung, 8kg"
    },
    {
        "product_id": 9,
        "product_name": "mesin cuci LG 10kg",
        "tags": "mesin cuci, LG, 10kg"
    },
]

## Membaca dan Memahami Dataset dari data yang diberikan

In [13]:
import pandas as pd


df = pd.DataFrame(data)
print(df.head())

  product_id        product_name                    tags
0          1  samsung galaxy s21  HP, samsung, handphone
1          2           iphone 12    HP, apple, handphone
2          3      google pixel 6   HP, google, handphone
3          4        xiaomi mi 11   HP, xiaomi, handphone
4          5           oneplus 9  HP, oneplus, handphone


## Preprocessing Data
Lakukan preprocessing data genre kita sebelum bisa menggunakannya untuk rekomendasi.

In [14]:
# Menghapus nilai NaN dari kolom 'tags'
df['tags'] = df['tags'].fillna('')

# Menyusun data tags sebagai kombinasi string
df['tags'] = df['tags'].apply(lambda x: ' '.join(x.split(', ')))

print(df['tags'].head())

0    HP samsung handphone
1      HP apple handphone
2     HP google handphone
3     HP xiaomi handphone
4    HP oneplus handphone
Name: tags, dtype: object


## Ekstraksi Fitur
Kita akan menggunakan CountVectorizer untuk mengubah genre film menjadi fitur vektor.


In [15]:
from sklearn.feature_extraction.text import CountVectorizer

# Membuat vektor dari tags
count = CountVectorizer()
count_matrix = count.fit_transform(df['tags'])

print(count_matrix.shape)

(9, 16)


## Menghitung Similaritas
Kita akan menggunakan cosine similarity untuk menghitung kemiripan antara film berdasarkan genre.

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

# Menghitung cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.66666667, 0.66666667, 0.66666667, 0.66666667,
        0.        , 0.33333333, 0.28867513, 0.        ],
       [0.66666667, 1.        , 0.66666667, 0.66666667, 0.66666667,
        0.        , 0.33333333, 0.        , 0.        ],
       [0.66666667, 0.66666667, 1.        , 0.66666667, 0.66666667,
        0.        , 0.33333333, 0.        , 0.        ],
       [0.66666667, 0.66666667, 0.66666667, 1.        , 0.66666667,
        0.        , 0.33333333, 0.        , 0.        ],
       [0.66666667, 0.66666667, 0.66666667, 0.66666667, 1.        ,
        0.        , 0.33333333, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.33333333, 0.        , 0.        ],
       [0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333,
        0.33333333, 1.        , 0.        , 0.        ],
       [0.28867513, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        , 0.5       ],


## Membuat Fungsi Rekomendasi
Selanjutnya, kita akan membuat fungsi untuk merekomendasikan film berdasarkan genre yang diberikan.

In [26]:
# Membuat mapping dari product_id ke indeks
indices = pd.Series(df.index, index=df['product_id']).drop_duplicates()

def get_recommendations(product_id, cosine_sim=cosine_sim):
    # Mendapatkan indeks produk yang diberikan
    idx = indices[product_id]

    # Mendapatkan skor similarity dari semua produk dengan produk yang diberikan
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Mengurutkan produk berdasarkan skor similarity
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Mendapatkan skor dari 10 produk yang paling mirip
    sim_scores = sim_scores[1:11]

    # Mendapatkan indeks produk
    product_indices = [i[0] for i in sim_scores]

    # Mengembalikan 10 produk yang paling mirip
    return df.iloc[product_indices]

# Contoh penggunaan
recommendations = get_recommendations('1')
print(recommendations[['product_id', 'product_name', 'tags']])

  product_id             product_name                    tags
1          2                iphone 12      HP apple handphone
2          3           google pixel 6     HP google handphone
3          4             xiaomi mi 11     HP xiaomi handphone
4          5                oneplus 9    HP oneplus handphone
6          7          HP Spectre x360       laptop HP spectre
7          8   mesin cuci samsung 8kg  mesin cuci samsung 8kg
5          6  Laptop Lenovo Ideapad 3   laptop lenovo ideapad
8          9       mesin cuci LG 10kg      mesin cuci LG 10kg


## Menguji Sistem Rekomendasi
Coba beberapa film untuk melihat apakah sistem rekomendasi bekerja dengan baik.

In [27]:
print(get_recommendations('1')[['product_id', 'product_name', 'tags']])
print(get_recommendations('8')[['product_id', 'product_name', 'tags']])

  product_id             product_name                    tags
1          2                iphone 12      HP apple handphone
2          3           google pixel 6     HP google handphone
3          4             xiaomi mi 11     HP xiaomi handphone
4          5                oneplus 9    HP oneplus handphone
6          7          HP Spectre x360       laptop HP spectre
7          8   mesin cuci samsung 8kg  mesin cuci samsung 8kg
5          6  Laptop Lenovo Ideapad 3   laptop lenovo ideapad
8          9       mesin cuci LG 10kg      mesin cuci LG 10kg
  product_id             product_name                   tags
8          9       mesin cuci LG 10kg     mesin cuci LG 10kg
0          1       samsung galaxy s21   HP samsung handphone
1          2                iphone 12     HP apple handphone
2          3           google pixel 6    HP google handphone
3          4             xiaomi mi 11    HP xiaomi handphone
4          5                oneplus 9   HP oneplus handphone
5          6  L